In [12]:
from sudoku import Sudoku
import pandas as pd
import subprocess
import tempfile

In [13]:
#get main cnf file
path = 'sudoku9.cnf'
cnf =""
f = open(path, 'r')
for line in f:
    cnf+= line
f.close()

In [14]:
# Global Constants
MIN_VARS = 1
MAX_VARS = 25
INSTANCES_PER_VAR = 100
SUDOKU_DEGREE = 9
VAR_COUNT = int(str(SUDOKU_DEGREE) + str(SUDOKU_DEGREE) + str(SUDOKU_DEGREE))

In [15]:
frames = [] # will store dataframes for each variable

In [17]:
frames = []
for var in range(MIN_VARS, MAX_VARS):
    boards = []
    frame = pd.DataFrame(columns=['board', 'solution'])
    for y in range(INSTANCES_PER_VAR):
        board = Sudoku(SUDOKU_DEGREE)
        board.randomize_board(var)
        boards.append(board)
    cnfs = [board.cnf_output() for board in boards]
    frame['board'] = boards
    frame['cnf'] = cnfs
    frame['num_vars'] = var
    frames.append(frame)
frames = pd.DataFrame(pd.concat(frames))
frames.head(5)

,board,solution,cnf,num_vars
0,<sudoku.Sudoku object at 0x7ff84ca875f8>,NaN,628 0\n,1
1,<sudoku.Sudoku object at 0x7ff84ca87630>,NaN,266 0\n,1
2,<sudoku.Sudoku object at 0x7ff84ca876a0>,NaN,669 0\n,1
3,<sudoku.Sudoku object at 0x7ff84ca877b8>,NaN,409 0\n,1
4,<sudoku.Sudoku object at 0x7ff84ca877f0>,NaN,721 0\n,1


In [18]:
def get_solution_from_cnf(cnf_string):
    
    cnf_string = cnf + cnf_string #append the whole cnf
    
    bashCommand = 'minisat %s %s' # command to run on the terminal
    
     # Our temporary input/output files 
    input_file = tempfile.NamedTemporaryFile(mode="w")
    output_file = tempfile.NamedTemporaryFile()
    
    # Count number of clauses
    clause_count = len(cnf_string.splitlines())
    
    # writes the file with DIMACS suitable header
    input_file.write("p cnf " + str(VAR_COUNT) + " " + str(clause_count) + "\n" + cnf_string)
 
    # call the bash terminal
    subprocess.call(bashCommand % (input_file.name, output_file.name), shell=True)
    
    input_file.close()
    
    return output_file.read().decode("utf-8") 

In [19]:
%%time
frames['solution'] = frames['cnf'].apply(get_solution_from_cnf)

CPU times: user 2.54 s, sys: 4.8 s, total: 7.33 s
Wall time: 26.1 s


In [20]:
frames.head(5)

,board,solution,cnf,num_vars
0,<sudoku.Sudoku object at 0x7ff84ca875f8>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,628 0\n,1
1,<sudoku.Sudoku object at 0x7ff84ca87630>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,266 0\n,1
2,<sudoku.Sudoku object at 0x7ff84ca876a0>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,669 0\n,1
3,<sudoku.Sudoku object at 0x7ff84ca877b8>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,409 0\n,1
4,<sudoku.Sudoku object at 0x7ff84ca877f0>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,721 0\n,1
